In [125]:
import parse

In [292]:
from aocd.models import Puzzle

puzzle = Puzzle(year=2021, day=18)

def parses(text):
    return [json.loads(line) for line in text.strip().split('\n')]

data = parses(puzzle.input_data)

In [293]:
from dataclasses import dataclass

In [17]:
from typing import Union

In [314]:
x = SN.fromlist([1,[2,[3,4]]])
x

[1, [2, [3, 4]]]

In [621]:
from heapq import heappop, heappush

@dataclass
class SN:
    left: Union['SN', int]
    right: Union['SN', int]
        
    def __repr__(self):
        return f'[{repr(self.left)},{repr(self.right)}]'
    
    @staticmethod
    def fromlist(data):
        l, r = data
        left = l if isinstance(l, int) else SN.fromlist(l)
        right = r if isinstance(r, int) else SN.fromlist(r)
        return SN(left, right)
    
    def __add__(self, other):
        return SN(self, other).reduce()
    
    def __getitem__(self, pos):
        head, tail = pos[0], pos[1:]
        node = [self.left, self.right][head]
        return node if len(tail) == 0 else node[tail]
    
    def __setitem__(self, pos, val):
        head, tail = pos[0], pos[1:]
        if len(tail) == 0:
            if head == 0:
                self.left = val
            else:
                self.right = val
        else:
            node = [self.left, self.right][head]
            node[tail] = val
            
    def find_neighbor(self, pos, side):
        # 0 is left, 1 is right
        if len(pos) == 0:
            return None
        if pos[-1] == 1-side:
            neighbor = pos[:-1] + (side,)
            while not isinstance(self[neighbor], int):
                neighbor += (1-side,)
            return neighbor
        return self.find_neighbor(pos[:-1], side)
            
    def neighbors(self, pos):
        return self.find_neighbor(pos, 0), self.find_neighbor(pos, 1)
    
    def __iter__(self):
        return iter([self.left, self.right])
    
    def find_explode(self, explode):
        stack = [(self, tuple())]
        while stack:
            num, chain = stack.pop()
            if len(chain) == 4:
                heappush(explode, chain)
            for i, child in enumerate((num.left, num.right)):
                c = chain + (i,)
                if not isinstance(child, int):
                    stack.append((child, c))
        return explode
    
    def find_split(self):
        stack = [(self, tuple())]
        while stack:
            num, pos = stack.pop()         
            if isinstance(num, int) and num > 9:
                return pos
            elif isinstance(num, SN):
                for i, child in enumerate((num.right, num.left)):
                    c = pos + (1-i,)
                    stack.append((child, c))
        return None
    
    def reduce(self):
        explode = []
        self.find_explode(explode)
        
        while True:
#             printc(explode, color='RED')
#             printc(split, color='GREEN')
#             printc(self, color='BLUE')
            if explode:
                pos = heappop(explode)
                pair = (self[pos].left, self[pos].right)
                self[pos] = 0
                for neigh, val in zip(self.neighbors(pos), pair):
                    if neigh is not None:
                        self[neigh] = self[neigh] + val
#                         if self[neigh] > 9 and len(neigh) <= 4:
#                             heappush(split, neigh)
#                 print('exploded:', self)
                continue
            
            to_split = self.find_split()
            
            if to_split is not None:
                pos = to_split
#                 pos = heappop(split)
#                 print(pos)
                val = self[pos]
                n = val//2
                self[pos] = SN(n, val-n)    
                if len(pos) == 4:
                    heappush(explode, pos)
#                 print('splitted:', self)
                continue
            break
        return self

    def magnitude(self):
        left = self.left if isinstance(self.left, int) else self.left.magnitude()
        right = self.right if isinstance(self.right, int) else self.right.magnitude()
        return 3*left + 2*right

In [619]:
from heapq import heappop, heappush

@dataclass
class SN:
    left: Union['SN', int]
    right: Union['SN', int]
        
    def __repr__(self):
        return f'[{repr(self.left)},{repr(self.right)}]'
    
    @staticmethod
    def fromlist(data):
        l, r = data
        left = l if isinstance(l, int) else SN.fromlist(l)
        right = r if isinstance(r, int) else SN.fromlist(r)
        return SN(left, right)
    
    def __add__(self, other):
        return SN(self, other).reduce()
    
    def __getitem__(self, pos):
        head, tail = pos[0], pos[1:]
        node = [self.left, self.right][head]
        return node if len(tail) == 0 else node[tail]
    
    def __setitem__(self, pos, val):
        head, tail = pos[0], pos[1:]
        if len(tail) == 0:
            if head == 0:
                self.left = val
            else:
                self.right = val
        else:
            node = [self.left, self.right][head]
            node[tail] = val
            
    def find_neighbor(self, pos, side):
        # 0 is left, 1 is right
        if len(pos) == 0:
            return None
        if pos[-1] == 1-side:
            neighbor = pos[:-1] + (side,)
            while not isinstance(self[neighbor], int):
                neighbor += (1-side,)
            return neighbor
        return self.find_neighbor(pos[:-1], side)
            
    def neighbors(self, pos):
        return self.find_neighbor(pos, 0), self.find_neighbor(pos, 1)
    
    def __iter__(self):
        return iter([self.left, self.right])
    
    def find_explode(self):
        explode = []
        stack = [(self, tuple())]
        while stack:
            num, chain = stack.pop()
            if len(chain) == 4:
                heappush(explode, chain)
            for i, child in enumerate((num.left, num.right)):
                c = chain + (i,)
                if not isinstance(child, int):
                    stack.append((child, c))
        return explode
    
#     def find_split(self):
#         stack = [(self, tuple())]
#         while stack:
#             num, pos = stack.pop()         
#             if isinstance(num, int) and num > 9:
#                 return pos
#             elif isinstance(num, SN):
#                 for i, child in enumerate((num.right, num.left)):
#                     c = pos + (1-i,)
#                     stack.append((child, c))
#         return None
    
    def reduce(self):
        explode = self.find_explode()
        split = []
        
        while True:
#             print(split)
#             printc(explode, color='RED')
#             printc(split, color='GREEN')
#             printc(self, color='BLUE')
            if explode:
                pos = heappop(explode)
                pair = (self[pos].left, self[pos].right)
                self[pos] = 0
                for neigh, val in zip(self.neighbors(pos), pair):
                    if neigh is not None:
                        if self[neigh] < 10 and self[neigh] + val >= 10:
                            heappush(split, neigh)
                        self[neigh] = self[neigh] + val
#                         if self[neigh] > 9 and len(neigh) <= 4:
#                             heappush(split, neigh)
#                 print('exploded:', self)
                continue
                        
            if split:
                pos = heappop(split)
#                 pos = heappop(split)
#                 print(pos)
                try:
                    val = self[pos]
                except TypeError:
                    val = None
                if isinstance(val, int):
                    n = val//2
                    self[pos] = SN(n, val-n)
                    if len(pos) == 4:
                        heappush(explode, pos)
                    else:
                        if n > 9:
                            heappush(split, pos+(0,))
                        if val-n > 9:
                            heappush(split, pos+(1,))
#                     print('splitted:', self)
                continue
            break
        return self

    def magnitude(self):
        left = self.left if isinstance(self.left, int) else self.left.magnitude()
        right = self.right if isinstance(self.right, int) else self.right.magnitude()
        return 3*left + 2*right

In [649]:
from heapq import heappop, heappush

@dataclass
class SN:
    left: Union['SN', int]
    right: Union['SN', int]
        
    def __repr__(self):
        return f'[{repr(self.left)},{repr(self.right)}]'
    
    @staticmethod
    def fromlist(data):
        l, r = data
        left = l if isinstance(l, int) else SN.fromlist(l)
        right = r if isinstance(r, int) else SN.fromlist(r)
        return SN(left, right)
    
    def __add__(self, other):
        return SN(self, other).reduce()
    
    def __getitem__(self, pos):
        head, tail = pos[0], pos[1:]
        node = [self.left, self.right][head]
        return node if len(tail) == 0 else node[tail]
    
    def __setitem__(self, pos, val):
        head, tail = pos[0], pos[1:]
        if len(tail) == 0:
            if head == 0:
                self.left = val
            else:
                self.right = val
        else:
            node = [self.left, self.right][head]
            node[tail] = val
            
    def find_neighbor(self, pos, side):
        # 0 is left, 1 is right
        if len(pos) == 0:
            return None
        if pos[-1] == 1-side:
            neighbor = pos[:-1] + (side,)
            while not isinstance(self[neighbor], int):
                neighbor += (1-side,)
            return neighbor
        return self.find_neighbor(pos[:-1], side)
            
    def neighbors(self, pos):
        return self.find_neighbor(pos, 0), self.find_neighbor(pos, 1)
    
    def __iter__(self):
        return iter([self.left, self.right])
    
    def find_explode(self):
        stack = [(self, tuple())]
        while stack:
            num, pos = stack.pop()
            if len(pos) == 4:
                return pos
            for i, child in enumerate((num.right, num.left)):
                if isinstance(child, SN):
                    stack.append((child, pos + (1-i,)))
    
    def find_split(self):
        stack = [(self, tuple())]
        while stack:
            num, pos = stack.pop()         
            if isinstance(num, int) and num > 9:
                return pos
            elif isinstance(num, SN):
                for i, child in enumerate((num.right, num.left)):
                    stack.append((child, pos + (1-i,)))
                    
    def reduce(self):
        
        while True:
#             printc(explode, color='RED')
#             printc(split, color='GREEN')
#             printc(self, color='BLUE')
            if pos := self.find_explode():
                pair = (self[pos].left, self[pos].right)
                self[pos] = 0
                for neigh, val in zip(self.neighbors(pos), pair):
                    if neigh is not None:
                        self[neigh] = self[neigh] + val
#                         if self[neigh] > 9 and len(neigh) <= 4:
#                             heappush(split, neigh)
#                 print('exploded:', self)
                continue
        
            if pos := self.find_split():
#                 pos = heappop(split)
#                 print(pos)
                val = self[pos]
                n = val//2
                self[pos] = SN(n, val-n)    
#                 print('splitted:', self)
                continue
    
            break
        return self

    def magnitude(self):
        left = self.left if isinstance(self.left, int) else self.left.magnitude()
        right = self.right if isinstance(self.right, int) else self.right.magnitude()
        return 3*left + 2*right

In [650]:
def solve_b(nums):
    C = SN.fromlist
    return max((C(a)+C(b)).magnitude() for a, b in itertools.product(nums,repeat=2) if a != b)

In [651]:
solve_b(data)

4695

In [652]:
%%timeit
solve_b(data)

5.33 s ± 39.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [646]:
a = SN.fromlist([[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]])
b = SN.fromlist([7,[[[3,7],[4,3]],[[6,3],[8,8]]]])
a + b

(0, 0, 0, 1)
(0, 1, 0, 0)
(0, 1, 0, 1)
(1, 1, 0, 0)
(1, 1, 0, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)
(0, 1, 1, 0)
(0, 1, 1, 1)
(1, 1, 0, 0)
(1, 1, 0, 1)
(1, 1, 1, 0)
(1, 1, 1, 1)


[[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]]

In [610]:
a = SN.fromlist([[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]])
b = SN.fromlist([7,[[[3,7],[4,3]],[[6,3],[8,8]]]])
c = SN.fromlist([[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]])
d = SN.fromlist([[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]])
e = SN.fromlist([7,[5,[[3,8],[1,4]]]])
a+b #+c #+d+e

[[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]]

[7,0]

In [605]:
a = SN.fromlist([[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]])
b = SN.fromlist([7,[[[3,7],[4,3]],[[6,3],[8,8]]]])
c = SN.fromlist([[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]])
d = SN.fromlist([[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]])
e = SN.fromlist([7,[5,[[3,8],[1,4]]]])
a+b #+c #+d+e

[]
exploded: [[[[4,0],[5,0]],[[[4,5],[2,6]],[9,5]]],[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]]
[]
exploded: [[[[4,0],[5,4]],[[0,[7,6]],[9,5]]],[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]]
[]
exploded: [[[[4,0],[5,4]],[[7,0],[15,5]]],[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]]
[(0, 1, 1, 0)]
exploded: [[[[4,0],[5,4]],[[7,0],[15,5]]],[10,[[0,[11,3]],[[6,3],[8,8]]]]]
[(0, 1, 1, 0), (1, 0), (1, 1, 0, 1, 0)]
exploded: [[[[4,0],[5,4]],[[7,0],[15,5]]],[10,[[11,0],[[9,3],[8,8]]]]]
[(0, 1, 1, 0), (1, 0), (1, 1, 0, 1, 0), (1, 1, 0, 0)]
exploded: [[[[4,0],[5,4]],[[7,0],[15,5]]],[10,[[11,9],[0,[11,8]]]]]
[(0, 1, 1, 0), (1, 0), (1, 1, 0, 1, 0), (1, 1, 0, 0), (1, 1, 1, 1, 0)]
exploded: [[[[4,0],[5,4]],[[7,0],[15,5]]],[10,[[11,9],[11,0]]]]
[(0, 1, 1, 0), (1, 0), (1, 1, 0, 1, 0), (1, 1, 0, 0), (1, 1, 1, 1, 0), (1, 1, 1, 0)]
splitted: [[[[4,0],[5,4]],[[7,0],[[7,8],5]]],[10,[[11,9],[11,0]]]]
[(1, 0), (1, 1, 0, 0), (1, 1, 0, 1, 0), (1, 1, 1, 0), (1, 1, 1, 1, 0)]
exploded: [[[[4,0],[5,4]],[[7,7],[0,13]]],[10,[[11,9],[11,0]]]]
[(0, 

[[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]]

In [ ]:
[[[[0, 7], 4], [[7, 8], [6, 0]]], [8, 1]]

In [442]:
L = [
    [1,1],
    [2,2],
    [3,3],
    [4,4],
    [5,5],
    [6,6],
]

In [443]:
from functools import reduce
from operator import add

In [444]:
reduce(add, map(SN.fromlist, L))

[[[[5,0],[7,4]],[5,5]],[6,6]]

In [237]:
def parses(text):
    return [SN.parse(line) for line in text.strip().split('\n')]

In [454]:
a = SN.fromlist([[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]])
b = SN.fromlist([7,[[[3,7],[4,3]],[[6,3],[8,8]]]])
c = SN.fromlist([[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]])
d = SN.fromlist([[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]])
e = SN.fromlist([7,[5,[[3,8],[1,4]]]])
a+b #+c #+d+e

[[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]]

In [458]:
SN.fromlist([[[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]],[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]]).reduce()

exploded: [[[[0,[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]],[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]]
exploded: [[[[5,0],[[11,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]],[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]]
exploded: [[[[5,11],[0,[13,0]]],[[8,[7,7]],[[7,9],[5,0]]]],[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]]
exploded: [[[[5,11],[13,0]],[[8,[7,7]],[[7,9],[5,0]]]],[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]]
exploded: [[[[5,11],[13,0]],[[15,0],[[14,9],[5,0]]]],[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]]
exploded: [[[[5,11],[13,0]],[[15,14],[0,[14,0]]]],[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]]
exploded: [[[[5,11],[13,0]],[[15,14],[14,0]]],[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]]
exploded: [[[[5,11],[13,0]],[[15,14],[14,0]]],[[2,[0,[11,4]]],[[[6,7],1],[7,[1,6]]]]]
exploded: [[[[5,11],[13,0]],[[15,14],[14,0]]],[[2,[11,0]],[[[10,7],1],[7,[1,6]]]]]
exploded: [[[[5,11],[13,0]],[[15,14],[14,0]]],[[2,[11,10]],[[0,8],[7,[1,6]]]]]
exploded: [[[[5,11],[13,0]],[[15,14],[14,0]]],[[2,[11,10]],[[

[[[[7,7],[7,0]],[[7,7],[7,7]]],[[[7,7],[8,8]],[[5,8],[8,0]]]]

In [584]:
SN.fromlist([[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]]) + \
SN.fromlist([[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]])

TypeError: 'int' object is not subscriptable

In [ ]:
[[[[7,7],[7,0]],[[7,7],[7,7]]],[[[7,7],[8,8]],[[5,8],[8,0]]]]

In [509]:
sample = parses("""[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]
[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]
[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]
[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]
[7,[5,[[3,8],[1,4]]]]
[[2,[2,2]],[8,[8,1]]]
[2,9]
[1,[[[9,3],9],[[9,0],[0,7]]]]
[[[5,[7,4]],7],1]
[[[[4,2],2],6],[8,7]]""")

In [490]:
reduce(add, map(SN.fromlist, sample))

[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]

In [510]:
sample = parses("""[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]
""")

In [492]:
reduce(add, map(SN.fromlist, sample))

[[[[6,6],[7,6]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[7,8],[9,9]]]]

In [606]:
def solve_a(nums):
    return reduce(add, map(SN.fromlist, nums)).magnitude()

In [607]:
solve_a(sample)

[]
exploded: [[[[5,0],[[9,7],[9,6]]],[[4,[1,2]],[[1,4],2]]],[[[5,[2,8]],4],[5,[[9,9],0]]]]
[]
exploded: [[[[5,9],[0,[16,6]]],[[4,[1,2]],[[1,4],2]]],[[[5,[2,8]],4],[5,[[9,9],0]]]]
[(0, 0, 1, 1, 0)]
exploded: [[[[5,9],[16,0]],[[10,[1,2]],[[1,4],2]]],[[[5,[2,8]],4],[5,[[9,9],0]]]]
[(0, 0, 1, 0), (0, 0, 1, 1, 0), (0, 1, 0, 0)]
exploded: [[[[5,9],[16,0]],[[11,0],[[3,4],2]]],[[[5,[2,8]],4],[5,[[9,9],0]]]]
[(0, 0, 1, 0), (0, 0, 1, 1, 0), (0, 1, 0, 0)]
exploded: [[[[5,9],[16,0]],[[11,3],[0,6]]],[[[5,[2,8]],4],[5,[[9,9],0]]]]
[(0, 0, 1, 0), (0, 0, 1, 1, 0), (0, 1, 0, 0)]
exploded: [[[[5,9],[16,0]],[[11,3],[0,6]]],[[[7,0],12],[5,[[9,9],0]]]]
[(0, 0, 1, 0), (0, 0, 1, 1, 0), (0, 1, 0, 0), (1, 0, 1)]
exploded: [[[[5,9],[16,0]],[[11,3],[0,6]]],[[[7,0],12],[14,[0,9]]]]
[(0, 0, 1, 0), (0, 0, 1, 1, 0), (0, 1, 0, 0), (1, 0, 1), (1, 1, 0)]
splitted: [[[[5,9],[[8,8],0]],[[11,3],[0,6]]],[[[7,0],12],[14,[0,9]]]]
[(0, 0, 1, 1, 0), (1, 0, 1), (0, 1, 0, 0), (1, 1, 0)]
exploded: [[[[5,17],[0,8]],[[11,3],[0,6]]]

[(0, 1, 0, 1), (0, 1, 1, 0, 0), (0, 1, 1, 0), (1, 0, 0, 0), (0, 1, 1, 1), (0, 1, 1, 1, 0)]
exploded: [[[[0,8],[15,0]],[[9,10],[12,16]]],[[[21,0],[8,8]],[[0,6],9]]]
[(0, 0, 1, 0), (0, 1, 1, 0, 0), (0, 1, 0, 1), (1, 0, 0, 0), (0, 1, 1, 1), (0, 1, 1, 1, 0), (0, 1, 1, 0)]
splitted: [[[[0,8],[[7,8],0]],[[9,10],[12,16]]],[[[21,0],[8,8]],[[0,6],9]]]
[(0, 1, 0, 1), (0, 1, 1, 0, 0), (0, 1, 1, 0), (1, 0, 0, 0), (0, 1, 1, 1), (0, 1, 1, 1, 0)]
exploded: [[[[0,15],[0,8]],[[9,10],[12,16]]],[[[21,0],[8,8]],[[0,6],9]]]
[(0, 0, 0, 1), (0, 1, 1, 0, 0), (0, 1, 0, 1), (1, 0, 0, 0), (0, 1, 1, 1), (0, 1, 1, 1, 0), (0, 1, 1, 0)]
splitted: [[[[0,[7,8]],[0,8]],[[9,10],[12,16]]],[[[21,0],[8,8]],[[0,6],9]]]
[(0, 1, 0, 1), (0, 1, 1, 0, 0), (0, 1, 1, 0), (1, 0, 0, 0), (0, 1, 1, 1), (0, 1, 1, 1, 0)]
exploded: [[[[7,0],[8,8]],[[9,10],[12,16]]],[[[21,0],[8,8]],[[0,6],9]]]
[(0, 1, 0, 1), (0, 1, 1, 0, 0), (0, 1, 1, 0), (1, 0, 0, 0), (0, 1, 1, 1), (0, 1, 1, 1, 0)]
splitted: [[[[7,0],[8,8]],[[9,[5,5]],[12,16]]],[[[21,0],

splitted: [[[[6,6],[7,7]],[[7,7],[0,7]]],[[[7,7],[8,8]],[[8,8],[7,[10,11]]]]]
[]
exploded: [[[[6,6],[7,7]],[[7,7],[0,7]]],[[[7,7],[8,8]],[[8,8],[17,0]]]]
[(1, 1, 1, 0)]
splitted: [[[[6,6],[7,7]],[[7,7],[0,7]]],[[[7,7],[8,8]],[[8,8],[[8,9],0]]]]
[]
exploded: [[[[6,6],[7,7]],[[7,7],[0,7]]],[[[7,7],[8,8]],[[8,16],[0,9]]]]
[(1, 1, 0, 1)]
splitted: [[[[6,6],[7,7]],[[7,7],[0,7]]],[[[7,7],[8,8]],[[8,[8,8]],[0,9]]]]
[]
exploded: [[[[6,6],[7,7]],[[7,7],[0,7]]],[[[7,7],[8,8]],[[16,0],[8,9]]]]
[(1, 1, 0, 0)]
splitted: [[[[6,6],[7,7]],[[7,7],[0,7]]],[[[7,7],[8,8]],[[[8,8],0],[8,9]]]]
[]
exploded: [[[[6,6],[7,7]],[[7,7],[0,7]]],[[[7,7],[8,16]],[[0,8],[8,9]]]]
[(1, 0, 1, 1)]
splitted: [[[[6,6],[7,7]],[[7,7],[0,7]]],[[[7,7],[8,[8,8]]],[[0,8],[8,9]]]]
[]
exploded: [[[[6,6],[7,7]],[[7,7],[0,7]]],[[[7,7],[16,0]],[[8,8],[8,9]]]]
[(1, 0, 1, 0)]
splitted: [[[[6,6],[7,7]],[[7,7],[0,7]]],[[[7,7],[[8,8],0]],[[8,8],[8,9]]]]
[]
exploded: [[[[6,6],[7,7]],[[7,7],[0,7]]],[[[7,15],[0,8]],[[8,8],[8,9]]]]
[(1, 0, 0, 

[(1, 0, 0, 0), (1, 1, 1, 0), (1, 0, 1, 1)]
splitted: [[[[7,7],[7,7]],[[0,8],[7,8]]],[[[[7,8],0],[7,18]],[[9,9],[10,0]]]]
[(1, 0, 1, 1), (1, 1, 1, 0)]
exploded: [[[[7,7],[7,7]],[[0,8],[7,15]]],[[[0,8],[7,18]],[[9,9],[10,0]]]]
[(0, 1, 1, 1), (1, 1, 1, 0), (1, 0, 1, 1)]
splitted: [[[[7,7],[7,7]],[[0,8],[7,[7,8]]]],[[[0,8],[7,18]],[[9,9],[10,0]]]]
[(1, 0, 1, 1), (1, 1, 1, 0)]
exploded: [[[[7,7],[7,7]],[[0,8],[14,0]]],[[[8,8],[7,18]],[[9,9],[10,0]]]]
[(0, 1, 1, 0), (1, 1, 1, 0), (1, 0, 1, 1)]
splitted: [[[[7,7],[7,7]],[[0,8],[[7,7],0]]],[[[8,8],[7,18]],[[9,9],[10,0]]]]
[(1, 0, 1, 1), (1, 1, 1, 0)]
exploded: [[[[7,7],[7,7]],[[0,15],[0,7]]],[[[8,8],[7,18]],[[9,9],[10,0]]]]
[(0, 1, 0, 1), (1, 1, 1, 0), (1, 0, 1, 1)]
splitted: [[[[7,7],[7,7]],[[0,[7,8]],[0,7]]],[[[8,8],[7,18]],[[9,9],[10,0]]]]
[(1, 0, 1, 1), (1, 1, 1, 0)]
exploded: [[[[7,7],[7,7]],[[7,0],[8,7]]],[[[8,8],[7,18]],[[9,9],[10,0]]]]
[(1, 0, 1, 1), (1, 1, 1, 0)]
splitted: [[[[7,7],[7,7]],[[7,0],[8,7]]],[[[8,8],[7,[9,9]]],[[9,9],[10,0

exploded: [[[[6,6],[7,7]],[[0,7],[7,8]]],[[[20,0],[24,0]],[[19,0],[9,4]]]]
[(1, 0, 0, 0), (1, 1, 0, 0), (1, 0, 1, 0)]
splitted: [[[[6,6],[7,7]],[[0,7],[7,8]]],[[[[10,10],0],[24,0]],[[19,0],[9,4]]]]
[(1, 0, 1, 0), (1, 1, 0, 0)]
exploded: [[[[6,6],[7,7]],[[0,7],[7,18]]],[[[0,10],[24,0]],[[19,0],[9,4]]]]
[(0, 1, 1, 1), (1, 0, 0, 1), (1, 0, 1, 0), (1, 1, 0, 0)]
splitted: [[[[6,6],[7,7]],[[0,7],[7,[9,9]]]],[[[0,10],[24,0]],[[19,0],[9,4]]]]
[(1, 0, 0, 1), (1, 1, 0, 0), (1, 0, 1, 0)]
exploded: [[[[6,6],[7,7]],[[0,7],[16,0]]],[[[9,10],[24,0]],[[19,0],[9,4]]]]
[(0, 1, 1, 0), (1, 0, 0, 1), (1, 0, 1, 0), (1, 1, 0, 0)]
splitted: [[[[6,6],[7,7]],[[0,7],[[8,8],0]]],[[[9,10],[24,0]],[[19,0],[9,4]]]]
[(1, 0, 0, 1), (1, 1, 0, 0), (1, 0, 1, 0)]
exploded: [[[[6,6],[7,7]],[[0,15],[0,8]]],[[[9,10],[24,0]],[[19,0],[9,4]]]]
[(0, 1, 0, 1), (1, 0, 0, 1), (1, 0, 1, 0), (1, 1, 0, 0)]
splitted: [[[[6,6],[7,7]],[[0,[7,8]],[0,8]]],[[[9,10],[24,0]],[[19,0],[9,4]]]]
[(1, 0, 0, 1), (1, 1, 0, 0), (1, 0, 1, 0)]
exploded

4140

In [521]:
def solve_b(nums):
    C = SN.fromlist
    return max((C(a)+C(b)).magnitude() for a, b in itertools.product(nums,repeat=2) if a != b)

In [522]:
solve_b(sample)

3993

In [523]:
solve_b(data)

4695

In [9]:
SN.parse('[[[[4,3],4],4],[7,[[8,4],9]]]')

[[[[4, 3], 4], 4], [7, [[8, 4], 9]]]

In [3]:
print(data)

[[8,[6,[0,1]]],[8,[2,[1,9]]]]
[[[5,[1,7]],9],[4,[[9,4],4]]]
[[[1,[6,3]],8],[[3,[5,3]],[1,6]]]
[0,[[1,[7,4]],[6,[7,3]]]]
[[[7,6],2],[[[9,5],9],[[5,5],[6,3]]]]
[[3,[[2,3],0]],0]
[4,[2,[4,[2,6]]]]
[[2,[8,[4,5]]],[[[7,7],7],1]]
[[[[6,7],6],[9,8]],[[0,[7,3]],[[9,1],[2,0]]]]
[0,[[[8,4],[4,9]],5]]
[[0,7],[8,5]]
[6,[[9,[0,7]],[[0,0],[8,1]]]]
[[[[8,3],[1,9]],[[9,3],[6,5]]],4]
[[[6,6],[[1,2],[1,7]]],[[8,8],[3,2]]]
[[[6,4],[[0,3],1]],[[6,2],[4,[0,3]]]]
[[[2,9],[[2,1],1]],[[6,[1,4]],[6,[0,3]]]]
[9,[[7,4],[1,9]]]
[[[[1,2],[7,8]],[[9,6],[1,3]]],[[0,6],[[3,6],6]]]
[[[1,[8,6]],[2,[3,4]]],[[0,4],[5,[5,7]]]]
[[[5,9],[[1,0],[4,3]]],[[7,[6,7]],[1,[1,5]]]]
[[[[6,8],[5,9]],[5,[4,5]]],4]
[[[[3,4],4],7],[[5,[3,7]],7]]
[[[[3,3],[7,9]],[1,[4,8]]],0]
[[[3,[9,4]],[1,[3,7]]],[[[1,8],1],[6,1]]]
[3,[[[7,5],[4,8]],[7,8]]]
[[7,[2,[2,4]]],[0,[8,[0,3]]]]
[[[[8,5],3],[3,[8,3]]],[1,[0,[7,4]]]]
[[[[7,1],3],[3,4]],[[3,7],[[1,8],[4,8]]]]
[[[3,[9,9]],4],[[4,2],[[4,2],4]]]
[[[5,[9,1]],[[3,5],[1,9]]],7]
[[[[0,8],5],[9,[5,1]]],[

In [128]:
def sign(x):
    if x == 0:
        return 0
    elif x > 0:
        return 1
    else:
        return -1

In [129]:
sample = parses("target area: x=20..30, y=-10..-5")

In [130]:
def solve_a(bounds):
    vxs = range(1,right)
    vys = range(abs(bottom),bottom,-1)
    maxy = float('-inf')
    best_init = None
    for vx, vy in itertools.product(vxs, vys):
        x, y = 0, 0
        init = (vx, vy)
        local_maxy = 0
        for t in itertools.count():
            x = x + vx
            y = y + vy
            local_maxy = max(local_maxy, y)
            vx -= sign(vx)
            vy -= 1
            if left <= x <= right and bottom <= y <= top:
                return local_maxy
            if ((x < left and vx == 0) or 
                (x > right and vx > 0) or
                (y < bottom)):
                break        

In [131]:
solve_a(sample)

45

In [132]:
solve_a(data)

9870

In [133]:
def solve_b(bounds):
    left, right, bottom, top = bounds
    vxs = range(1,right+1)
    vys = range(10*(top-bottom),bottom-1,-1)
    ways = 0
    for vx, vy in itertools.product(vxs, vys):
        x, y = 0, 0
        init = (vx, vy)
        local_maxy = 0
        for t in itertools.count():
            x = x + vx
            y = y + vy
            local_maxy = max(local_maxy, y)
            vx -= sign(vx)
            vy -= 1
            if left <= x <= right and bottom <= y <= top:
                ways += 1
                break
            if ((x < left and vx == 0) or 
                (x > right and vx > 0) or
                (y < bottom)):
                break       
    return ways

In [134]:
solve_b(sample)

112

In [135]:
solve_b(data)

5523

In [ ]:
(left+right)//2

In [140]:
fastsolve_a(sample)

45

In [141]:
fastsolve_a(data)

9870

In [167]:
[i*(i+1)//2 for i in range(1,10)]

[1, 3, 6, 10, 15, 21, 28, 36, 45]

In [204]:
5+4+3+1

13

In [218]:
bounds = (42,42,-10,-1)  
print(solve_a(bounds)) 
print(fastsolve_a(bounds))

3
45


In [ ]:
bounds = (10,10,-10,-1)  
print(solve_a(bounds)) 
print(fastsolve_a(bounds))

In [219]:
bounds = (45,45,-9,0)  
print(solve_a(bounds)) 
print(fastsolve_a(bounds))

36
36


In [213]:
tricky_sample = (42,42,-10,-1)

In [ ]:
def f(x):
    2*x

In [ ]:
x^2 + x - 2c = 0

In [188]:
def triangular_in_interval(low, high):
    # Solve for (x*(x+1))/2 = triangular
    a = math.ceil((-1 + math.sqrt(1+8*low))/2)
    b = math.floor((-1 + math.sqrt(1+8*high))/2)
    return a == b

In [ ]:
def solve_a(bounds):
    left, right, bottom, top = bounds
    # x positions are triangular number, can only reach valid 
    if not triangular_in_interval(low, high):
        

    max_vy = 0-bottom # We reach the bottom in a last step from 0
    return max_vy*(max_vy-1)//2

In [190]:
triangular_in_interval(16,21)

True

In [179]:
triangular_in_interval(16,19)

True

In [176]:
f(16)

(5.178908345800274, -6.178908345800274)

In [216]:
def valid_velocities(bounds):
    left, right, bottom, top = bounds
    vxs = range(1, right + 1)
    vys = range(abs(bottom), bottom - 1, -1)
    for vx, vy in itertools.product(vxs, vys):
        x, y = 0, 0
        init = (vx, vy)
        local_maxy = 0
        while not ((x < left and vx == 0) or (x > right and vx > 0) or (y < bottom)):
            x = x + vx
            y = y + vy
            vx -= sign(vx)
            vy -= 1
            local_maxy = max(local_maxy, y)
            if left <= x <= right and bottom <= y <= top:
                yield (init, local_maxy)
                break


In [217]:
next(valid_velocities((45,45,-9,0)))

((9, 9), 45)